<a href="https://colab.research.google.com/github/simoneVU/CGAN/blob/main/Background_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing required libraries

In [12]:
#Installing libraries for background removal
!pip install rembg[gpu,cli]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.0 MB/s eta 0:00

## Background Removal

In [13]:
#Checking how to use the tool from the terminal, however, it will be mainly used as library
!rembg --help

Usage: rembg [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  b  for a byte stream as input
  i  for a file as input
  p  for a folder as input
  s  for a http server


### Basic Background Removal
The background of the image is removed with the default ML model.

In [15]:
from rembg import remove

def remove_background(input_image_path, output_image_path):
  with open(input_image_path, 'rb') as i:
    with open(output_image_path, 'wb') as o:
        input = i.read()
        output = remove(input)
        o.write(output)

In [16]:
input_path = '/content/drive/MyDrive/AntonUpWork/background_removal/image.png'
output_path = '/content/drive/MyDrive/AntonUpWork/background_removal/image_without_background.png'

remove_background(input_path, output_path)


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 150GB/s]


### Background Removal with specific model
Choose the ML model to use to remove the image baground to improve it quality

In [17]:
from PIL import Image
from rembg import new_session, remove

input = Image.open(input_path)

The models you can choose form are the following:


  - **u2net**: A pre-trained model for general use cases.
  - **u2netp**: A lightweight version of u2net model.
  - **u2net_human_seg**: A pre-trained model for human segmentation.
  - **u2net_cloth_seg**: A pre-trained model for Cloths Parsing from human portrait. Here clothes are parsed into 3 category: Upper body, Lower body and Full body.
  - **silueta**: Same as u2net but the size is reduced to 43Mb.
  - **isnet-general-use** : A new pre-trained model for general use cases.
  - **isnet-anime**: A high-accuracy segmentation for anime character.
  - **sam**: A pre-trained model for any use cases.


In [18]:
model_name = "u2net_human_seg"
session = new_session(model_name)
output = remove(input, session=session)

100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 35.8GB/s]


In [19]:
#Use alpha matting for a better quality
output = remove(input, session=session, alpha_matting=True, alpha_matting_foreground_threshold=270,alpha_matting_background_threshold=20, alpha_matting_erode_size=11)

### Paste image back with new background

In [20]:
def custom_background(background_file, foreground):
  final_foreground = Image.fromarray(foreground)
  background = Image.open(background_file)
  # Resize background image to match the size of the foreground image
  background = background.resize(final_foreground.size, Image.LANCZOS)

  x = (background.size[0]-final_foreground.size[0])/2 + 0.5
  y = (background.size[1]-final_foreground.size[1])/2 + 0.5
  box = (x, y, final_foreground.size[0] + x, final_foreground.size[1] + y)
  crop = background.crop(box)
  final_image = crop.copy()
  # put the foreground in the centre of the background
  paste_box = (0, final_image.size[1] - final_foreground.size[1], final_image.size[0], final_image.size[1])
  final_image.paste(final_foreground, paste_box, mask=final_foreground)
  return final_image

In [21]:
import matplotlib.pyplot as plt
import numpy as np

background_file = '/content/drive/MyDrive/AntonUpWork/background_removal/background_2.jpg'
foreground_file = np.array(Image.open(output_path))

final_image = custom_background(background_file, foreground_file)

final_image.save('/content/drive/MyDrive/AntonUpWork/background_removal/background_merged_background.jpg')

### Enhance image lighiting

Cloning into 'Illumination-Adaptive-Transformer'...
remote: Enumerating objects: 2523, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 2523 (delta 47), reused 0 (delta 0), pack-reused 2432
Receiving objects: 100% (2523/2523), 30.07 MiB | 14.24 MiB/s, done.
Resolving deltas: 100% (863/863), done.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
